In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import pandas as pd
import datetime as dt
import re
from unidecode import unidecode
from betting_functions import get_url_json

In [2]:
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
data = get_url_json(dog_props_url)

In [3]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [4]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
998,aa304d69-8e50-44da-9b0b-014672125dd8,False,None,[{'id': '84b119f8-620b-4096-9732-e90d383cf594'...,85471a2a-d5a6-4c3d-8f8b-229a1cbaca85,25143021,43.5,active,85471a2a-d5a6-4c3d-8f8b-229a1cbaca85,a5fa5201-36ad-4fd0-97cf-93fd0a3afc52,e9b627d8-cd29-4a94-ae00-bdd26ad09da5,Kills on Maps 1+2+3,high_score,cd401dc8-c89b-45c8-a879-faec2762d590,kills_on_maps_1_2_3,None,None,Val: Chadi Kills on Maps 1+2+3 O/U
1397,bd9fc552-9029-4d86-aec1-493fd9735898,False,None,[{'id': '4b9340d1-948f-4633-b380-5d3ebd7999a0'...,4ba1df0b-c98a-403e-9c19-be5d821c928f,25190029,24.0,active,4ba1df0b-c98a-403e-9c19-be5d821c928f,d6a6e182-7f7e-43b3-85e0-7dd74988698e,3d0578fb-e276-4ffe-874f-81052348a2d7,Assists on Maps 1+2+3,high_score,22661db9-40ca-454d-a5ed-916872e82175,assists_on_maps_1_2_3,None,None,LoL: StanPelly Assists on Maps 1+2+3 O/U


In [5]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [6]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [7]:
player_df

,full_name,sport_id,team_id
0,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
1,Damian Lillard,NBA,e142b705-dd0a-4803-8cde-a35788a65341
2,Brook Lopez,NBA,e142b705-dd0a-4803-8cde-a35788a65341
3,Tyrese Haliburton,NBA,4da4781a-a2c2-49d7-a042-1dee71740668
4,Myles Turner,NBA,4da4781a-a2c2-49d7-a042-1dee71740668
...,...,...,...
477,Jordan Mickey,BASKETBALL,abdc3a59-2734-4381-bbe1-9a3847df3e73
478,Isaia Cordinier,BASKETBALL,abdc3a59-2734-4381-bbe1-9a3847df3e73
491,T. Young + D. Murray,NBACOMBOS,37cbb4fc-d4f4-4afd-b2b9-940715fcf818
492,P. Banchero + F. Wagner,NBACOMBOS,14a6b030-ca78-4cca-911d-9622d29290bb


In [8]:
player_df.sport_id.unique()

array(['NBA', 'NFL', 'CFB', 'NHL', 'SOCCER', 'ESPORTS', 'BASKETBALL',
       'NBACOMBOS'], dtype=object)

In [9]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [10]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
49,57672.0,3e1f3d38-19a1-4e4e-b043-8e9375df6e8d,0.0,2af0b792-f585-4c49-aeb6-eafc61073a21,0.0,Thu 02:30pm,0.0,2023-11-09T19:30:00Z,regular,BASKETBALL,scheduled,Fenerbahce @ KK Partizan Nis Belgrade,Game,2023.0
50,57673.0,73019268-6016-4a30-8734-24935592408c,0.0,cbb6e581-5e37-4a3f-be13-ef5d9f932477,0.0,Thu 02:30pm,0.0,2023-11-09T19:30:00Z,regular,BASKETBALL,scheduled,Valencia Basket Club @ Milano,Game,2023.0
51,57674.0,bc9ff096-3676-4ba5-ad41-3d674a566ee1,0.0,abdc3a59-2734-4381-bbe1-9a3847df3e73,0.0,Thu 02:45pm,0.0,2023-11-09T19:45:00Z,regular,BASKETBALL,scheduled,Virtus Bologna @ Real Madrid,Game,2023.0


In [11]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [12]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [13]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [14]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,52473.0,e142b705-dd0a-4803-8cde-a35788a65341,0.0,4da4781a-a2c2-49d7-a042-1dee71740668,0.0,Thu 07:00pm,0.0,2023-11-09,regular,NBA,scheduled,MIL @ IND,Game,2023.0
1,52474.0,04fc51d6-320c-4838-a087-f492067b71f5,0.0,27c02dfc-343a-432b-8a64-c963035b609d,0.0,Thu 09:30pm,0.0,2023-11-09,regular,NBA,scheduled,ATL @ ORL,Game,2023.0
2,39443.0,17f3bc4a-e2d6-5dc5-a554-00549ff0139f,0.0,e8a4678a-ccdb-5f35-be13-aef6e59a5680,0.0,Thu 08:15pm,0.0,2023-11-09,regular,NFL,scheduled,CAR @ CHI,Game,2023.0
3,39444.0,f11f1cf1-9933-5203-8181-95020ee64399,0.0,ef876de9-81ac-5e60-af6e-0ca1700f3a51,0.0,Sun 09:30am,0.0,2023-11-12,regular,NFL,scheduled,IND @ NE,Game,2023.0
4,39450.0,7161e62b-de20-56e2-a300-0dc23637faaa,0.0,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,SF @ JAX,Game,2023.0
5,39448.0,338ae7df-02ad-5a94-9767-60f511bd55e1,0.0,01bfe9d5-f671-57ed-aa60-249fcca9267c,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,CLE @ BAL,Game,2023.0
6,39447.0,4ab08caa-b79d-598e-82ef-0906a60d2a89,0.0,ab235cf0-a041-5d36-8241-90a90f0dcb5e,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,HOU @ CIN,Game,2023.0
7,39446.0,530518ed-91db-57c4-9077-27cc0dd9a293,0.0,5af1e185-627e-5345-851d-3c65c5b66614,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,NO @ MIN,Game,2023.0
8,39445.0,a2fadaac-562f-5a7e-bd63-6d88d90c1ac4,0.0,ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,GB @ PIT,Game,2023.0
9,39449.0,f96aa8db-21c2-5b86-b49d-7e64b4eda61d,0.0,1a20f1da-c502-5224-9c4a-bc363174cd21,0.0,Sun 01:00pm,0.0,2023-11-12,regular,NFL,scheduled,TEN @ TB,Game,2023.0


In [15]:
id2tm

{'4da4781a-a2c2-49d7-a042-1dee71740668': 'IND',
 '27c02dfc-343a-432b-8a64-c963035b609d': 'ORL',
 'e8a4678a-ccdb-5f35-be13-aef6e59a5680': 'CHI',
 'ef876de9-81ac-5e60-af6e-0ca1700f3a51': 'NE',
 'b49b653c-dc7c-516e-b1c2-da1b30d1b1a6': 'JAX',
 '01bfe9d5-f671-57ed-aa60-249fcca9267c': 'BAL',
 'ab235cf0-a041-5d36-8241-90a90f0dcb5e': 'CIN',
 '5af1e185-627e-5345-851d-3c65c5b66614': 'MIN',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 '1a20f1da-c502-5224-9c4a-bc363174cd21': 'TB',
 'c7b497d4-18b6-522b-abaa-e5c3d24bc021': 'LAC',
 '8459772c-695a-5890-afa1-7ec38da17201': 'ARI',
 '31631011-5902-52f6-ba01-c4c8d8eb3fd9': 'SEA',
 '26e67e06-664e-50a6-ad7b-c102705fde8b': 'DAL',
 '99b6f0b0-3970-4c2b-aba5-f18804aff3fd': 'ULL',
 '9f3952f8-b2c5-4531-bcd3-1dbd970250ff': 'SMU',
 'fc1177f8-beed-4170-acf5-fe669bb4795a': 'COL',
 '185d72b0-fc5a-463f-925b-203748f37b70': 'NYR',
 '5cdace81-efe8-4b20-ada3-e9ceb0d52048': 'TB',
 'cd01a7e8-34d4-4add-941e-84560138d229': 'OTT',
 'c285294d-d222-4072-b965-9b2daec69679': 'D

In [16]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [17]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Giannis Antetokounmpo', 'Damian Lillard', 'Brook Lopez',
       'Tyrese Haliburton', 'Myles Turner', 'Bruce Brown',
       'Bennedict Mathurin', 'Obi Toppin', 'Trae Young',
       'Dejounte Murray', 'Clint Capela', "De'Andre Hunter",
       'Bogdan Bogdanovic', 'Jalen Johnson', 'Paolo Banchero',
       'Franz Wagner', 'Jalen Suggs', 'Bryce Young', 'Miles Sanders',
       'Chuba Hubbard', 'Adam Thielen', 'Jonathan Mingo', 'Hayden Hurst',
       'Terrace Marshall Jr.', 'Eddy Pineiro', 'Tommy Tremble',
       'Frankie Luvu', 'Derrick Brown', 'Donte Jackson', 'Xavier Woods',
       'Tyson Bagent', 'Roschon Johnson', "D'Onta Foreman", 'DJ Moore',
       'Darnell Mooney', 'Cole Kmet', 'Robert Tonyan', 'Cairo Santos',
       'Eddie Jackson', 'T.J. Edwards', 'Jaylon Johnson', 'Jack Sanborn',
       'Tyrique Stevenson', 'Jaquan Brisker', 'Gardner Minshew II',
       'Jonathan Taylor', 'Michael Pittman Jr.', 'Mac Jones',
       'Rhamondre Stevenson', 'Ezekiel Elliott', 'Brock Purdy',
   

In [18]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NBA', 'NBA', 'NBA', 'NBA', 'NBA']

In [19]:
guys[1000:1007]

['Val: PROFEK',
 'Val: Zeek',
 'Val: TajReX',
 'Val: luckyMrJ',
 'Val: xad',
 'Val: Luvjix',
 'Val: krejzzs']

In [20]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341',
 'e142b705-dd0a-4803-8cde-a35788a65341']

In [21]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,44.5,pts_rebs_asts,Giannis Antetokounmpo Pts + Rebs + Asts O/U,Giannis Antetokounmpo,NBA
1,5.0,assists,Giannis Antetokounmpo Assists O/U,Giannis Antetokounmpo,NBA
2,11.0,rebounds,Giannis Antetokounmpo Rebounds O/U,Giannis Antetokounmpo,NBA
3,16.5,rebs_asts,Giannis Antetokounmpo Rebounds + Assists O/U,Giannis Antetokounmpo,NBA
4,0.5,three_points_made,Giannis Antetokounmpo 3-Pointers Made O/U,Giannis Antetokounmpo,NBA
...,...,...,...,...,...
1659,68.5,pts_rebs_asts,T. Young + D. Murray Pts + Rebs + Asts O/U,T. Young + D. Murray,other
1660,42.5,points,P. Banchero + F. Wagner Points O/U,P. Banchero + F. Wagner,other
1661,62.5,pts_rebs_asts,P. Banchero + F. Wagner Pts + Rebs + Asts O/U,P. Banchero + F. Wagner,other
1662,24.5,points,C. Capela + D. Hunter Points O/U,C. Capela + D. Hunter,other


In [22]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,44.5,pts_rebs_asts,Giannis Antetokounmpo Pts + Rebs + Asts O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
1,5.0,assists,Giannis Antetokounmpo Assists O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
2,11.0,rebounds,Giannis Antetokounmpo Rebounds O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
3,16.5,rebs_asts,Giannis Antetokounmpo Rebounds + Assists O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
4,0.5,three_points_made,Giannis Antetokounmpo 3-Pointers Made O/U,Giannis Antetokounmpo,NBA,e142b705-dd0a-4803-8cde-a35788a65341
...,...,...,...,...,...,...
1659,68.5,pts_rebs_asts,T. Young + D. Murray Pts + Rebs + Asts O/U,T. Young + D. Murray,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818
1660,42.5,points,P. Banchero + F. Wagner Points O/U,P. Banchero + F. Wagner,other,14a6b030-ca78-4cca-911d-9622d29290bb
1661,62.5,pts_rebs_asts,P. Banchero + F. Wagner Pts + Rebs + Asts O/U,P. Banchero + F. Wagner,other,14a6b030-ca78-4cca-911d-9622d29290bb
1662,24.5,points,C. Capela + D. Hunter Points O/U,C. Capela + D. Hunter,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818


In [23]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [24]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,44.5,pts_rebs_asts,Giannis Antetokounmpo Pts + Rebs + Asts O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-09
1,5.0,assists,Giannis Antetokounmpo Assists O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-09
2,11.0,rebounds,Giannis Antetokounmpo Rebounds O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-09
3,16.5,rebs_asts,Giannis Antetokounmpo Rebounds + Assists O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-09
4,0.5,three_points_made,Giannis Antetokounmpo 3-Pointers Made O/U,Giannis Antetokounmpo,NBA,MIL,2023-11-09
...,...,...,...,...,...,...,...
1659,68.5,pts_rebs_asts,T. Young + D. Murray Pts + Rebs + Asts O/U,T. Young + D. Murray,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818,NaN
1660,42.5,points,P. Banchero + F. Wagner Points O/U,P. Banchero + F. Wagner,other,14a6b030-ca78-4cca-911d-9622d29290bb,NaN
1661,62.5,pts_rebs_asts,P. Banchero + F. Wagner Pts + Rebs + Asts O/U,P. Banchero + F. Wagner,other,14a6b030-ca78-4cca-911d-9622d29290bb,NaN
1662,24.5,points,C. Capela + D. Hunter Points O/U,C. Capela + D. Hunter,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818,NaN


In [25]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [26]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,44.5,pts_rebs_asts,Giannis Antetokounmpo,NBA,MIL,2023-11-09
1,5.0,assists,Giannis Antetokounmpo,NBA,MIL,2023-11-09
2,11.0,rebounds,Giannis Antetokounmpo,NBA,MIL,2023-11-09
3,16.5,rebs_asts,Giannis Antetokounmpo,NBA,MIL,2023-11-09
4,0.5,three_points_made,Giannis Antetokounmpo,NBA,MIL,2023-11-09
...,...,...,...,...,...,...
1659,68.5,pts_rebs_asts,T. Young + D. Murray,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818,NaN
1660,42.5,points,P. Banchero + F. Wagner,other,14a6b030-ca78-4cca-911d-9622d29290bb,NaN
1661,62.5,pts_rebs_asts,P. Banchero + F. Wagner,other,14a6b030-ca78-4cca-911d-9622d29290bb,NaN
1662,24.5,points,C. Capela + D. Hunter,other,37cbb4fc-d4f4-4afd-b2b9-940715fcf818,NaN


In [27]:
df = df.reset_index(drop=True)

In [28]:
df.Stat.unique()

array(['pts_rebs_asts', 'assists', 'rebounds', 'rebs_asts',
       'three_points_made', 'pts_rebs', 'pts_asts', 'personal_fouls',
       'blks_stls', 'field_goals_att', 'points', 'turnovers',
       'three_points_att', 'steals', 'blocks', 'passing_yds',
       'rushing_yds', 'passing_att', 'passing_comps',
       'passing_and_rushing_yds', 'rushing_att', 'receiving_yds',
       'rush_rec_yds', 'receiving_rec', 'extra_points_made',
       'kicking_points', 'tackles', 'tackles_and_assists', 'passing_ints',
       'rush_rec_tds', 'passing_tds', 'sacks', 'field_goals_made',
       'goals', 'shots', 'power_play_points', 'saves',
       'goals_and_assists', 'chances_created', 'goals_against', 'aces',
       'double_faults', 'sets_lost', 'fantasy_points',
       'service_games_lost', 'games_won', 'games_lost', 'games_played',
       'strokes', 'kills_in_map_1', 'kills_in_maps_1_2_3',
       'kills_in_map_3', 'kills_in_map_2', 'fantasy_points_on_maps_1_2_3',
       'fantasy_points_on_map_1', '

In [29]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [30]:
df = df.replace(syntax)

In [31]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
645,6.5,kills_in_map_1,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
646,5.5,kills_in_map_1,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
647,19.5,kills_in_maps_1_2_3,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
648,17.0,kills_in_maps_1_2_3,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
649,6.5,kills_in_map_3,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
...,...,...,...,...,...,...
1548,5.0,kills_in_maps_1_2,PolskiKoz,ESPORTS,Illuminar Gaming,2023-11-10
1549,6.0,kills_in_maps_1_2,Krysia,ESPORTS,Illuminar Gaming,2023-11-10
1550,7.0,kills_in_maps_1_2,Mrozku,ESPORTS,Illuminar Gaming,2023-11-10
1551,1.5,kills_in_maps_1_2,Syrpy,ESPORTS,Illuminar Gaming,2023-11-10


In [32]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [33]:
today = dt.date.today()

In [34]:
str(today)

'2023-11-09'

In [35]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/dog/doglines_{today.month}_{today.day}.csv')

In [36]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
645,6.5,kills_in_map_1,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
646,5.5,kills_in_map_1,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
647,19.5,kills_in_maps_1_2_3,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
648,17.0,kills_in_maps_1_2_3,Bin+XUN,ESPORTS,Bilibili Gaming,2023-11-11
649,6.5,kills_in_map_3,Elk+Yagao,ESPORTS,Bilibili Gaming,2023-11-11
...,...,...,...,...,...,...
1548,5.0,kills_in_maps_1_2,PolskiKoz,ESPORTS,Illuminar Gaming,2023-11-10
1549,6.0,kills_in_maps_1_2,Krysia,ESPORTS,Illuminar Gaming,2023-11-10
1550,7.0,kills_in_maps_1_2,Mrozku,ESPORTS,Illuminar Gaming,2023-11-10
1551,1.5,kills_in_maps_1_2,Syrpy,ESPORTS,Illuminar Gaming,2023-11-10
